In [1]:
from agents.email_agent import EmailAgent

a = EmailAgent()
a({'input':'test'})

/home/tp/smartlead_agent/agents/email_agent.py:51: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(
/home/tp/smartlead_agent/agents/email_agent.py:56: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  return initialize_agent(
/home/tp/smartlead_agent/agents/email_agent.py:18: LangChainDep



> Entering new AgentExecutor chain...
I'm here to assist you. How can I help you today?

> Finished chain.


{'input': 'test',
 'memory': [],
 'output': "I'm here to assist you. How can I help you today?"}